## Expectations with Busyness

### About expectations

There are different types of expectation rules which can be defined. Expectation can be fulfilled or violated when certain conditions are met or not met. An expectation which seems particularly relevant in an organ donor situation is a **within** rule - which states that following an event, another state will hold **within** a certain period of time. For example, we could have an expectation rule that states that a doctor must request a patient's medical history from their normal hospital or GP with a countdown value of 2. Note that this does not necessarily mean within 2 time periods of the patient's admission, but that when the patient is admitted, the expectation will hold with a countdown value of 2, and we expect this value to drop overtime. The expectation is fulfilled when the medical history has been requested before the countdown value drops below zero, and is violated if the medical history is not requested before the countdown value drops below zero.

### Set up

In [1]:
?- cd('~/work'), ['dec'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
false.
true.

### Busyness levels


This notebook exhibits how user defined busyness levels for the hospital can be defined, and how these can be used to affect expectation rules. The hospital has certain expectations about the time delay between related actions occurring, such as how soon after a patient's admission a log of that admission should be made. When the hospital is busy (currently defined as a high number of patients admitted), we may wish to relax some expectations on promptness of medical staff actions, such as making a diagnosis from a blood test result, so that events are not viewed as being less trustworthy due to a short delay. For other events, we may never want to relax our expectations.

### Declaring the levels:

We use the current number of patients admitted to hospital as a measure of the hospital's business, although other measures such as number of logs made or messages sent to other hospitals could be used, and busyness could be declared on a smaller scale (e.g. for hospital departments) or a larger scale (e.g. across all hospitals in a district health board). 

Level information is set using the predicate levels/3, where the first argument is a list of the level names, the second argument is the break points between the levels and the third argument is the weighting assigned to different types of events under the levels. 

In this example, we declare that there are three levels; low, medium and high, with break points at 2 and 4. The break points are set very low for ease of illustration, and mean that if there are 2 or fewer patients currently admitted, the hospital's busyness is low, if there are between 2 (exclusive) and 4 (inclusive) patients, the hospital's busyness is medium, and if there are more than 4 patients, the hospital's busyness is high.

We also declare the relative weightings to be assigned to different types of events. Here, logging events are given a weighting of 1 under all three levels, while **actorDec** events (representing actor decisions) are given a weighting of 1 under the low level, 0.666 under the medium level and 0.4 under the high level.

In [2]:
% File: levels.pl
levels([low,medium,high],[2,4],[logging=[1,1,1], actorDec=[1,0.6666666,0.4]]).

The event weightings are used to affect how far to progress expectations under a given busyness level. If an expectation containing a *within* rule states that a logging event will take place, and has a starting countdown value of 3, we expect the event to take place within the next three time periods, as logging events (and any events without a type specified) have a weighting of 1. Note that the starting countdown value is not necessarily the number of time periods before an expectation will be violated if the expected event does not take place. If an expectation with a *within* rule instead stated that an **actorDec** event will take place and has a start time of 3, we would expect the event to take place within 8 time periods under a high busyness level, as 3 / 0.4 = 8. The weightings control how quickly the specified coutndown valuedrops, and allow the rate of decrease in countdown value associated with some types of expectations to be modified by busyness level.

In [3]:
%File: levelRules.pl
% If the count is greater than the maximum breakpoint, the busyness level is the greatest level.
holdsAt(busyness(Level), T):- aggregate_all(count, holdsAt(admitted(_),T), Count), levels(Names,BreakPoints,_), last(Names, Level), last(BreakPoints, P), Count > P.
% Else, check which breakpoints the count falls between, using 0 as the minimum breakpoint.
holdsAt(busyness(Level),T):-  aggregate_all(count, holdsAt(admitted(_),T), Count), levels(Names,BreakPoints,_), member(Level,Names), nth0(LIndex,Names,Level), nth0(LIndex, BreakPoints, P),  P >= Count, (LIndex > 0 -> (LIndex2 is LIndex - 1, nth0(LIndex2,BreakPoints,P2), P2 < Count); true).
progress(within(F1,T1,Type), within(F1,T2,Type),T):- holdsAt(busyness(Level),T), levels(Levels,_,DecayList), member(Type=Decay,DecayList), nth0(Index, Levels, Level), nth0(Index,Decay,D), T2 is T1 - D. 

### Expectation rules

For illustration purposes, we will make use of a few expectation rules. Here, we declare that all events which are not log events will be logged within a certain time period, which we give a starting time of 2. This actually means that a logging event should take place one time period after the event itself, as it takes another time period for the log to hold. We also declare that when an event is logged, there should be an expectation of the logging event, in order to flag any log events not corresponding to events which have recently occurred. Thirdly, we declare that each log event should use a different ID.

In [4]:
% File: logging.pl

% We expect that all events except logging events will be logged with a starting wait of 2.
initially(exp_rule(and([happ(X), condition(X \= log(_,_) )]), within(exist(_,logged(_,X,_)),2, logging))).
% We expect that when an event is logged, the expectation of the logging taking place holds.
initially(exp_rule(happ(log(_,X)), exp(_,_,_,within(exist(_,logged(_,X,_)),_,logging)))).
% We expect that a log ID has not already been used for another log entry.
initially(exp_rule(happ(log(ID,_)), not(logged(ID,_,_)))).

Our other expectation rules state that a patient history should have been requested shortly after a patient's admission, and that a history request should receive a response, with an initial countdown value of 4.

In [5]:
% File: OtherExp.pl
% We expect that a patient history will be requested within two units from of admission.
% This is declared to be an actorDec type within rule.
initially(exp_rule(happ(admission(Patient,_)), within(requested(_,'history',Patient,_,_)='NA',2, actorDec))).
% We expect that a history request will be answered within 4 units from periods.
initially(exp_rule(happ(request(Reference,'history',Patient,Source,Requester)), within(requested(Reference,'history',Patient,Source,Requester)='ANS',4))).

We need some supporting initiates and terminates clauses, as shown below.

In [6]:
% File: eventToFluent.pl
initiates(log(ID,X), logged(ID,X,T),T).
initiates(request(Reference,Type,Patient,Source,Requester),requested(Reference,Type,Patient,Source,Requester)='NA',_).
initiates(admission(PatID,_), admitted(PatID),_).
terminates(discharge(PatID,_), admitted(PatID),_).

### Event narrative

In our event narrative, doctors are identified using their initials for ease of readability. (Practically, numerical IDs would be used to avoid duplicates). Here, 'AH' and 'PS' are two medical staff members at the hospital.

Our event narrative is as follows:
- Patient 124 is admitted by 'AH' at time period 1
- A patient history request for patient 124 with referral code 2 is sent to Wellington by 'PS' at time period 2. Messages, including requests and responses, are sent with referral codes to establish a chain of message responses between different hospitals.
- Patient 501 is admitted by 'AH' at time period 2.
- Patient 502 is admitted by 'AH' at time period 3.
- Patient 503 is admitted by 'AH' at time period 5.
- Patient 504 is admitted by 'AH' at time period 7.
- Patient 501 is discharged by 'AH' at time period 9.
- At time period 2, we log the admission which occurred at time period 1 with an ID of 1.
- At time period 3, we log the history request which occurred at time period 2 with an ID of 2.
- At time period 3, we log the admission which occurred at time period 2 with an ID of 3.
- At time period 8, we log the admission which occurred at time period 7 with an ID of 4.
- At time period 10, we log the discharge which occurred at time period 9 with an ID of 5.



In [7]:
% File: narrative.pl
happensAtNarrative(admission(124,'AH'),1).
happensAtNarrative(request(2,'history',124,'Wellington', 'PS'),2).
happensAtNarrative(admission(501,'AH'),2).
happensAtNarrative(admission(502,'AH'),3).
happensAtNarrative(admission(503,'AH'),5).
happensAtNarrative(admission(504,'AH'),7).
happensAtNarrative(discharge(501,'AH'),9).

happensAtNarrative(log(1, admission(124,'AH')),2).
happensAtNarrative(log(2, request(2, history, 123, 'Wellington', 'PS')),3).
happensAtNarrative(log(3, admission(501,'AH')),3).
happensAtNarrative(log(4, admission(504,'AH')),8).
happensAtNarrative(log(5,discharge(501,'AH')),10).

### Expectations under "Low" busyness

We record all the fluents that hold from time period 0-11. We can see that at time periods 0, 1 and 2, there are 2 or fewer patients admitted in the hospital, so the busyness level is low.

In [8]:
?- run(10).
?- holdsAt(busyness(Level1),1).
?- aggregate_all(count, holdsAt(admitted(_),1), Count1).
?- holdsAt(busyness(Level2),2).
?- aggregate_all(count, holdsAt(admitted(_),2), Count2).
?- holdsAt(busyness(Level3),3).
?- aggregate_all(count, holdsAt(admitted(_),3), Count3).


true.
Level1 = low .
Count1 = 0 .
Level2 = low .
Count2 = 1 .
Level3 = low .
Count3 = 2 .

Recall that patient 124 was admitted at time period 1. Under the low busyness setting, the countdown value for the expecation that a patient history request for patient 124 will be made decays at a rate of one unit per time period, and is fulfilled at time period 3.

In [9]:
?- holdsAt(exp(Exp1),1), Exp1 = within(requested(_,history,124,_,_)='NA',_,actorDec).
?- holdsAt(exp(Exp2),2), Exp2 = within(requested(_,history,124,_,_)='NA',_,actorDec).
?- holdsAt(exp(Exp3),3), Exp3 = within(requested(_,history,124,_,_)='NA',_,actorDec).
?- happensAt(fulf(_,_,_,within(requested(_,history,124,_,_)='NA',_,actorDec)),3).

Exp1 = within(=(requested(_44, history, 124, _50, _52), NA), 2, actorDec) .
Exp2 = within(=(requested(_44, history, 124, _50, _52), NA), 1, actorDec) .
Exp3 = within(=(requested(_44, history, 124, _50, _52), NA), 0, actorDec) .
true.

Likewise, the expectation that the admission of patient 124 will be logged decays at a rate of one unit per time period and is also fulfilled at time period 3.

In [10]:
?- holdsAt(exp(Exp1),1), Exp1 = within(exist(_, logged(_, admission(124,_),_)), _, logging).
?- holdsAt(exp(Exp2),2), Exp2 = within(exist(_, logged(_, admission(124,_),_)), _, logging).
?- holdsAt(exp(Exp3),3), Exp3 = within(exist(_, logged(_, admission(124,_),_)), _, logging).
?- happensAt(fulf(_,_,_,within(exist(_, logged(_, admission(124,_),_)), _, logging)),3).

Exp1 = within(exist(_60, logged(_52, admission(124, AH), _56)), 2, logging) .
Exp2 = within(exist(_60, logged(_52, admission(124, AH), _56)), 1, logging) .
Exp3 = within(exist(_60, logged(_52, admission(124, AH), _56)), 0, logging) .
true.

Recall that at time period 3, patient 502 is admitted to hospital. At this time period, the hospital is at the "Low" level of busyness. The expectation of the admission being logged and the expectation of a patient history request being made for the admission both have a initial value of 2.

In [11]:
?- happensAt(Event, 3), Event=admission(_,_).
?- holdsAt(busyness(Level),3).
?- holdsAt(exp(Exp4),3), Exp4 = within(exist(_, logged(_, admission(502,_),_)), _, logging). 
?- holdsAt(exp(Exp4),3),Exp4 = (within(requested(_, history, 502, _,_)='NA',_,actorDec)).

Event = admission(502, AH) .
Level = low .
Exp4 = within(exist(_60, logged(_52, admission(502, AH), _56)), 2, logging) .
Exp4 = within(=(requested(_46, history, 502, _52, _54), NA), 2, actorDec) .

### Expectations under "Medium" busyness

At time period 4, there are 3 admitted patients in the hospital and the hospital's busyness is set to medium.

In [12]:
?- holdsAt(busyness(Level),4).
?- aggregate_all(count, holdsAt(admitted(_),4), Count).

Level = medium .
Count = 3 .

We see that the countdown associated with the expectation that patient 502's admission will be logged has dropped to 1, as logging events have a weighting of 1 under any busyness level. However, the countdown for the history request has droppped by only 0.666 to 1.333. This is because the weighting for actorDec related expectations under the "Medium" busyness level is 0.666.

In [13]:
?- holdsAt(exp(Exp4),4), Exp4 = within(exist(_, logged(_, admission(502,_),_)), _, logging). 
?- holdsAt(exp(Exp4),4), Exp4 = (within(requested(_, history, 502, _,_)='NA',_,actorDec)).

Exp4 = within(exist(_60, logged(_52, admission(502, AH), _56)), 1, logging) .
Exp4 = within(=(requested(_46, history, 502, _52, _54), NA), 1.3333334, actorDec) .

At time period five, the countdown for the logging event has again dropped by one (to zero), while the countdown for the patient history request has dropped by 0.666 to 0.666.

In [14]:
?- holdsAt(exp(Exp5),5), Exp5 = within(exist(_, logged(_, admission(502,_),_)), _, logging). 
?- holdsAt(exp(Exp5),5), Exp5 = (within(requested(_, history, 502, _,_)='NA',_,actorDec)).

Exp5 = within(exist(_60, logged(_52, admission(502, AH), _56)), 0, logging) .
Exp5 = within(=(requested(_46, history, 502, _52, _54), NA), 0.6666667999999999, actorDec) .

At time period 6, the countdown value for the logging expectation has dropped below zero. At this time period, our logging expectation has been violated. 

The countdown value for our patient history request expectation has not yet dropped below zero, so the expectation still holds.

In [15]:
?- holdsAt(exp(Exp6),6), Exp6 = within(exist(_, logged(_, admission(502,_),_)), _, logging). 
?- holdsAt(exp(Exp6),6), Exp6 = (within(requested(_, history, 502, _,_)='NA',_,actorDec)).
?- happensAt(viol(_,_,_,ViolEvent),6), ViolEvent = within(exist(_, logged(_, admission(502,_),_)), _, logging).

Exp6 = within(exist(_60, logged(_52, admission(502, AH), _56)), -1, logging) .
Exp6 = within(=(requested(_46, history, 502, _52, _54), NA), 1.9999999989472883e-07, actorDec) .
ViolEvent = within(exist(_70, logged(_62, admission(502, AH), _66)), -1, logging) .

At time period 7, the countdown value for the patient history request expectation has dropped below zero, so the expectation is violated.

In [16]:
?- holdsAt(exp(Exp7),7), Exp7 = (within(requested(_, history, 502, _,_)='NA',_,actorDec)).
?- happensAt(viol(_,_,_,ViolEvent),7), ViolEvent = (within(requested(_, history, 502, _,_)='NA',_,actorDec)).

Exp7 = within(=(requested(_46, history, 502, _52, _54), NA), -0.6666664000000001, actorDec) .
ViolEvent = within(=(requested(_56, history, 502, _62, _64), NA), -0.6666664000000001, actorDec) .

 At time period 8, there are 5 admitted patients in the hospital and the hospital's busyness is set to high. At time period 10, there are 4 admitted patients in the hospital and the hospital's busyness drops back to medium.

In [17]:
?- holdsAt(busyness(Level8),8).
?- aggregate_all(count, holdsAt(admitted(_),7), Count8).
?- holdsAt(busyness(Level10),10).
?- aggregate_all(count, holdsAt(admitted(_),10), Count10).

Level8 = high .
Count8 = 4 .
Level10 = medium .
Count10 = 4 .

As illustrated above, we are able to have an event trigger an expectation rule at one busyness level, and  the drop in the associated countdown value will be affected by the busyness level during future time periods, until the expectation is either fulfilled or violated. This allows us to extend the time period specified for **within** expectations under busy scenarios.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1527cc64-36a2-4b35-bd8b-8d493ca554fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>